# 1. Import libraries

In [1]:
import os
import time
from matplotlib import pyplot as plt
import numpy as np
from numpy import printoptions
import pandas as pd
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score
import torch
from torchvision import transforms
from torchvision import models
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
import random
import shutil
from datetime import datetime

In [2]:
import warnings
warnings.filterwarnings('always')

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True #Enable processing images(prevent OSError: image file is truncated)

# 2. Set paths
 - To load dataset
 - To save checkpoints & best checkpoints

In [3]:
#Set directories as you want.
path = "/home/ubuntu/Desktop/Project"
dataset_path = os.path.join(path, "datasets/circlin_feeds_dataset/image_dataset")

date = datetime.today().strftime("%Y%m%d")
print(f"Date today: {date}")
checkpoint_path = os.path.join(path, f"autolabeler_classifier/resnext50_model/{date}")
model_path = os.path.join(path, f"autolabeler_classifier/resnext50_model/{date}")
metric_path = os.path.join(path, f"autolabeler_classifier/resnext50_model/{date}")

# Save path for logs
# logdir = os.path.join(path, f"autolabeler_classifier/resnext50_model/{date}/logs")

Date today: 20211227


# 3. Training settings

## 3-1. Set seed number.

In [4]:
# Fix all seeds to make experiments reproducible
torch.manual_seed(2020)
torch.cuda.manual_seed(2020)
np.random.seed(2020)
random.seed(2020)
torch.backends.cudnn.deterministic = True

## 3-2. Hyperparameters
 - __Adjust: <u>mean</u>, <u>std</u>__

In [13]:
# Initialize the training parameters.s
NUM_WORKERS = 8 # Number of CPU processes for data preprocessing
LEARNING_RATE = 1e-5 # Learning rate
TRAIN_BATCH_SIZE = 256
VALID_BATCH_SIZE = 256
save_freq = 1 # Save checkpoint frequency (epochs)
test_freq = 200 # Test model frequency (iterations)
EPOCHS = 56 # Number of epochs for training 
# Note: on the small subset of data overfitting happens after 30-35 epochs


#For normalization
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]


# Run tensorboard
# %load_ext tensorboard
# %tensorboard --logdir {logdir}

## 3-3. Loss function

In [5]:
# Loss function
def loss_fn(outputs, targets):
    return torch.nn.BCELoss()(outputs, targets) #BCELoss()

## 3-4. Check GPU status & Enable distributed processing
 - __Should be improved!__ 
   - As is : Using DatParallel
   - To be: Use DistributedDataParallel

In [6]:
#Device check(for GPU computing)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [ ]:
#For multiple GPU utilization: This should be improved...

# dist.init_process_group(
#     backend='nccl',
#     init_method='tcp://localhost:9999', #FREEPORT
#     world_size=2,
#     rank=0,
# )

# dist.init_process_group(
#     backend="nccl",
#     init_method='tcp://127.0.0.1:9999',
#     rank=0,
#     world_size=2)

## 3-5. Optimizer

In [7]:
#Optimizer
def make_optimizer(model, lr):
    optimizer = torch.optim.Adam(
        params =  model.parameters(), 
        lr=lr)

    return optimizer

# 4. Prepare dataset

## 4-1. Define target labels(46)

In [8]:
#Define taret labels
labels = ['간편식', '건강간식', '건강식', '건강음료', '걷기/산책', '격투기', '골프', 
          '기타식단', '기타운동', '농구', '달리기/조깅', '당구', '등산/등반', '루틴기록', '맨몸', '무술', 
          '배구', '배드민턴', '보조제', '보충제', '볼링', '수상스포츠', '스키/스노보드', '승마', '신체기록', 
          '야구', '온라인클래스', '요가', '운동기구', '운동용품', '웨이트', '유산소기록', '의류', '일반간식', 
          '일반식', '일반음료', '일상생활', '자전거', '종합운동', '줄넘기', '축구/풋살', '탁구', '테니스', 
          '폴댄스', '필라테스', '홈트'] #46

## 4-2. Create custom dataset

- At Image.open in __ __getitem__ __  needs .convert('RGB') because Image.open returns grayscale.
    - https://stackoverflow.com/questions/59218671/runtimeerror-output-with-shape-1-224-224-doesnt-match-the-broadcast-shape

In [9]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, transforms):
        self.transforms = transforms
        self.df = df
        self.feed_image = df['url'] #Series of file name
        self.labels = self.df[labels].values #df.values: np.array #one-hot encoded: [0, 1, 0, ...., 1, 1]
        
        #self.image_list = self.feed_image.tolist()
        #self.label_list = self.labels.tolist()
        
    def __len__(self):
        return len(self.feed_image)

    def __getitem__(self, index):
        label = torch.FloatTensor(self.labels[index])
        image_url = self.feed_image[index]
        
        #Needs .convert('RGB') because Image.open returns grayscale.
        image = Image.open(image_url).convert('RGB')
        if self.transforms is not None:
            image = self.transforms(image)
            
        return image, label

    
# train_annotations = os.path.join(img_folder, 'small_train.json')
# train_dataset = CustomDataset(img_folder, train_annotations, train_transform)

In [10]:
#Get image dataset
dataset = os.path.join(dataset_path, "20211201_image_dataset(change_url).csv")
whole_df = pd.read_csv(dataset)
print(whole_df.columns)
print(whole_df['deidentification_x'].unique())
print(len(whole_df))

Index(['index', 'seq', 'url', 'deidentification_x', '간편식', '건강간식', '건강식',
       '건강음료', '걷기/산책', '격투기', '골프', '기타식단', '기타운동', '농구', '달리기/조깅', '당구',
       '등산/등반', '루틴기록', '맨몸', '무술', '배구', '배드민턴', '보조제', '보충제', '볼링', '수상스포츠',
       '스키/스노보드', '승마', '신체기록', '야구', '온라인클래스', '요가', '운동기구', '운동용품', '웨이트',
       '유산소기록', '의류', '일반간식', '일반식', '일반음료', '일상생활', '자전거', '종합운동', '줄넘기',
       '축구/풋살', '탁구', '테니스', '폴댄스', '필라테스', '홈트'],
      dtype='object')
['n']
215145


In [11]:
#Drop useless features/columns
copy_df = whole_df.copy()
copy_df.drop(labels=['index', 'seq', 'deidentification_x'], axis=1, inplace=True)
print(copy_df.columns)
copy_df.head(10)

Index(['url', '간편식', '건강간식', '건강식', '건강음료', '걷기/산책', '격투기', '골프', '기타식단',
       '기타운동', '농구', '달리기/조깅', '당구', '등산/등반', '루틴기록', '맨몸', '무술', '배구', '배드민턴',
       '보조제', '보충제', '볼링', '수상스포츠', '스키/스노보드', '승마', '신체기록', '야구', '온라인클래스',
       '요가', '운동기구', '운동용품', '웨이트', '유산소기록', '의류', '일반간식', '일반식', '일반음료',
       '일상생활', '자전거', '종합운동', '줄넘기', '축구/풋살', '탁구', '테니스', '폴댄스', '필라테스',
       '홈트'],
      dtype='object')


,url,간편식,건강간식,건강식,건강음료,걷기/산책,격투기,골프,기타식단,기타운동,...,일상생활,자전거,종합운동,줄넘기,축구/풋살,탁구,테니스,폴댄스,필라테스,홈트
0,/home/ubuntu/Desktop/Project/datasets/circlin_...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,/home/ubuntu/Desktop/Project/datasets/circlin_...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,/home/ubuntu/Desktop/Project/datasets/circlin_...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,/home/ubuntu/Desktop/Project/datasets/circlin_...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,/home/ubuntu/Desktop/Project/datasets/circlin_...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,/home/ubuntu/Desktop/Project/datasets/circlin_...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,/home/ubuntu/Desktop/Project/datasets/circlin_...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,/home/ubuntu/Desktop/Project/datasets/circlin_...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,/home/ubuntu/Desktop/Project/datasets/circlin_...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,/home/ubuntu/Desktop/Project/datasets/circlin_...,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
#Train - validation split
train_size = 0.8
train_df = copy_df.copy().sample(frac=train_size, random_state=200).reset_index(drop=True)
val_df = copy_df.drop(train_df.index).reset_index(drop=True)

# Train preprocessing
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(),
    transforms.RandomAffine(degrees=20, 
                            translate=(0.2, 0.2),
                            scale=(0.5, 1.5),
                            shear=None,
                            resample=False, 
                            fillcolor=tuple(np.array(np.array(mean)*255).astype(int).tolist())),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Test preprocessing
val_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
print(tuple(np.array(np.array(mean)*255).tolist()))

train_dataset = CustomDataset(train_df, train_transform)
valid_dataset = CustomDataset(val_df, val_transform)

(123.675, 116.28, 103.53)


/home/ubuntu/anaconda3/envs/tf-bert-text-classification/lib/python3.7/site-packages/torchvision/transforms/transforms.py:1362: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  "Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead"
/home/ubuntu/anaconda3/envs/tf-bert-text-classification/lib/python3.7/site-packages/torchvision/transforms/transforms.py:1376: UserWarning: Argument fillcolor is deprecated and will be removed since v0.10.0. Please, use fill instead
  "Argument fillcolor is deprecated and will be removed since v0.10.0. Please, use fill instead"


In [15]:
train_data_loader =  torch.utils.data.DataLoader(train_dataset, 
                              batch_size=TRAIN_BATCH_SIZE, 
                              num_workers=NUM_WORKERS,  #0?
                              shuffle=True,
                              drop_last=True)
val_data_loader =  torch.utils.data.DataLoader(valid_dataset, 
                             batch_size=VALID_BATCH_SIZE, 
                             num_workers=NUM_WORKERS) #0?

#num_train_batches = int(np.ceil(len(train_dataset) / batch_size))

In [16]:
# #To explore file shape.
# batchlist = []
# datalist = []
# for batch_idx, data in enumerate(train_data_loader):
#     #print(batch_idx, data)
#     batchlist.append(batch_idx)
#     datalist.append(data)

# 5. Make feed image classification model

## 5-1. Define functions that save checkpoint of model

In [17]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min #valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

## 5-2. Define Resnext50 model as a class
 - Use pytorch implemented pretrained model.

In [18]:
class ResNeXt50Class(nn.Module):
    def __init__(self, n_classes):
#         super().__init__()
#         resnet = models.resnext50_32x4d(pretrained=True)
#         resnet.fc = nn.Sequential(
#             nn.Dropout(p=0.2),
#             nn.Linear(in_features=resnet.fc.in_features, out_features=n_classes)
#         )
#         self.base_model = resnet
        super(ResNeXt50Class, self).__init__()
        self.resnext_model = models.resnext50_32x4d(pretrained=True)
        self.resnext_model.fc = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(in_features=self.resnext_model.fc.in_features, 
                      out_features=n_classes)
        )
        self.sigm = nn.Sigmoid()

    def forward(self, x):
        output = self.sigm(self.resnext_model(x))
        
        return output
        #return self.sigm(self.base_model(x))

In [19]:
model = ResNeXt50Class(len(labels)) #or len(labels) #train_dataset.classes
model = model.cuda()
model = nn.DataParallel(model) #Distributed
#model = nn.parallel.DistributedDataParallel(model, device_ids=[0, 1]) #Distributed DataParallel  ===> Should use this!!!!!!!!!!!!!!!!!
model.to(device)

DataParallel(
  (module): ResNeXt50Class(
    (resnext_model): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru

In [20]:
# optimizer
optimizer = make_optimizer(model, LEARNING_RATE)

In [21]:
val_targets = []
val_outputs = []

## 5-3. Training
 - __<u>Add Train Loss!!!!!!!!!!</u>__

In [22]:
# learning_rate = [
#                  0.00001, 0.00002, 0.00003, 0.00004, 0.00005,                 
#                  0.0001, 0.0002, 0.0003, 0.0004, 0.0005,
#                  0.001, 0.002, 0.003, 0.004, 0.005,
#                  0.01, 0.02, 0.03, 0.04, 0.05,
#                  0.1, 0.2, 0.3, 0.4, 0.5,
#                  0.000001, 0.000002, 0.000003, 0.000004, 0.000005]
train_losses_lr = {}
avg_train_losses_lr = {}
val_losses_lr = {}
avg_val_losses_lr = {}
epoch_list = [int(x) for x in np.linspace(1, EPOCHS, EPOCHS).tolist()]
print(epoch_list)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56]


In [23]:
def train_model(n_epochs,
                       training_loader,
                       validation_loader,
                       model,
                       optimizer,
                       checkpoint_path,
                       best_model_path,
                       metric_path,
                       date):
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf
    train_loss_epoch = []
    avg_train_loss_epoch = []    
    val_loss_epoch = [] #append to val_loss_list
    avg_val_loss_epoch = [] #append to avg_val_list
    
    for epoch in range(1, n_epochs+1):
        train_loss = 0
        valid_loss = 0

        model.train()
        print(f'############# Epoch {epoch}: Training Start   #############')
        for batch_idx, data in enumerate(training_loader):
            images, targets = data[0], data[1]
            images, targets = images.to(device), targets.to(device)
            
            outputs = model(images)

            optimizer.zero_grad()
            loss = loss_fn(outputs, targets)
            if batch_idx%5000==0:
                print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        print('############# Epoch {}: Training End     #############'.format(epoch))
        train_loss_epoch.append(train_loss)
        print('############# Epoch {}: Validation Start   #############'.format(epoch))
        ######################    
        # validate the model #
        ######################

        model.eval()
   
        with torch.no_grad():
            for batch_idx, data in enumerate(validation_loader, 0):
                images, targets = data[0], data[1]
                images, targets = images.to(device), targets.to(device)
                outputs = model(images)

                loss = loss_fn(outputs, targets)
                valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
                val_targets.extend(targets.cpu().detach().numpy().tolist())
                val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

            print('############# Epoch {}: Validation End     #############'.format(epoch))
           # calculate average losses
#             print('before calculate avg train loss', train_loss)
            val_loss_epoch.append(valid_loss) 
            avg_train_loss = train_loss/len(training_loader)
            avg_valid_loss = valid_loss/len(validation_loader)
            #Print training/validation statistics
            print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
                epoch, 
                avg_train_loss,
                avg_valid_loss
            ))
            avg_train_loss_epoch.append(avg_train_loss)
            avg_val_loss_epoch.append(avg_valid_loss) 
            

            # create checkpoint variable and add important data
            checkpoint = {
                    'epoch': epoch + 1,
                    'valid_loss_min': avg_valid_loss,
                    'state_dict': model.state_dict(),
                    'optimizer': optimizer.state_dict()
              }

            save_ckp(checkpoint, False,  f"{checkpoint_path}_{epoch}", best_model_path)
            
            ## TODO: save the model if validation loss has decreased
            if avg_valid_loss <= valid_loss_min:
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,avg_valid_loss))
                # save checkpoint as best model
                save_ckp(checkpoint, True,  f"{checkpoint_path}_{epoch}", best_model_path)
                valid_loss_min = avg_valid_loss

        now = datetime.today().strftime("%Y-%m-%d %H:%M:%S")
        log_text = f"[{now}]: [Learning Rate {lr}, Epoch {epoch}] - train_loss = {train_loss}, avg_train_loss = {avg_train_loss}, validation_loss = {valid_loss}, avg_validation_loss = {avg_valid_loss}\n"
        if os.path.isfile(os.path.join(metric_path, f"metric_logs_resnext_{date}.txt")):
            with open(os.path.join(metric_path, f"metric_logs_resnext_{date}.txt"), 'a', encoding='utf-8') as f:
                f.write(log_text)
        else:
            with open(os.path.join(metric_path, f"metric_logs_resnext_{date}.txt"), 'w', encoding='utf-8') as f:
                f.write(log_text)       
        print('############# Epoch {}  Done   #############\n'.format(epoch))

    train_losses_lr[lr] = train_loss_epoch
    avg_train_losses_lr[lr] = avg_train_loss_epoch
    val_losses_lr[lr] = val_loss_epoch
    avg_val_losses_lr[lr] = avg_val_loss_epoch
    print(f"train_losses_lr for LR {lr}: \n {train_losses_lr}")
    print(f"avg_train_losses_lr for LR {lr}: \n {avg_train_losses_lr}")
    print(f"val_losses_lr for LR {lr}: \n {val_losses_lr}")
    print(f"avg_val_losses_lr {lr}: \n {avg_val_losses_lr}")

    return model

In [24]:
# # Use threshold to define predicted labels and invoke sklearn's metrics with different averaging strategies.
# def calculate_metrics(pred, target, threshold=0.5):
#     pred = np.array(pred > threshold, dtype=float)
#     return {'micro/precision': precision_score(y_true=target, y_pred=pred, average='micro'),
#             'micro/recall': recall_score(y_true=target, y_pred=pred, average='micro'),
#             'micro/f1': f1_score(y_true=target, y_pred=pred, average='micro'),
#             'macro/precision': precision_score(y_true=target, y_pred=pred, average='macro'),
#             'macro/recall': recall_score(y_true=target, y_pred=pred, average='macro'),
#             'macro/f1': f1_score(y_true=target, y_pred=pred, average='macro'),
#             'samples/precision': precision_score(y_true=target, y_pred=pred, average='samples'),
#             'samples/recall': recall_score(y_true=target, y_pred=pred, average='samples'),
#             'samples/f1': f1_score(y_true=target, y_pred=pred, average='samples'),
#             }

### Set checkpoint path, best model's path.

In [25]:
ckpt_path = os.path.join(checkpoint_path, "curr_ckpt")
best_model_path = os.path.join(checkpoint_path, "best_model.pt")

### Training start!

In [ ]:
#For hyperparameter tuning
for lr in [LEARNING_RATE]:
    print('\n')
    print(f'##########################################################')
    print(f'##########################################################')    
    print(f'############### Training for learning rate {lr} START! ###############')
    print(f'##########################################################')
    print(f'##########################################################')
    print('\n')
    optimizer = make_optimizer(model, lr)
    train_model(EPOCHS,
               train_data_loader,
               val_data_loader,
               model,
               optimizer,
               os.path.join(checkpoint_path, f"curr_ckpt_{lr}"),
               best_model_path,
               metric_path,
               date)



##########################################################
##########################################################
############### Training for learning rate 1e-05 START! ###############
##########################################################
##########################################################


############# Epoch 1: Training Start   #############
Epoch: 1, Training Loss:  0.09049102663993835
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.000097 	Average Validation Loss: 0.000310
Validation loss decreased (inf --> 0.000310).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
Epoch: 2, Training Loss:  0.06278601288795471
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Va

Epoch: 18, Training Loss:  0.036217790096998215
############# Epoch 18: Training End     #############
############# Epoch 18: Validation Start   #############
############# Epoch 18: Validation End     #############
Epoch: 18 	Avgerage Training Loss: 0.000056 	Average Validation Loss: 0.000197
Validation loss decreased (0.000199 --> 0.000197).  Saving model ...
############# Epoch 18  Done   #############

############# Epoch 19: Training Start   #############
Epoch: 19, Training Loss:  0.03264780715107918
############# Epoch 19: Training End     #############
############# Epoch 19: Validation Start   #############
############# Epoch 19: Validation End     #############
Epoch: 19 	Avgerage Training Loss: 0.000055 	Average Validation Loss: 0.000195
Validation loss decreased (0.000197 --> 0.000195).  Saving model ...
############# Epoch 19  Done   #############

############# Epoch 20: Training Start   #############
Epoch: 20, Training Loss:  0.04001079127192497
############# Epoch 20

Epoch: 36, Training Loss:  0.025037916377186775
############# Epoch 36: Training End     #############
############# Epoch 36: Validation Start   #############
############# Epoch 36: Validation End     #############
Epoch: 36 	Avgerage Training Loss: 0.000042 	Average Validation Loss: 0.000157
Validation loss decreased (0.000159 --> 0.000157).  Saving model ...
############# Epoch 36  Done   #############

############# Epoch 37: Training Start   #############
Epoch: 37, Training Loss:  0.02692866139113903
############# Epoch 37: Training End     #############
############# Epoch 37: Validation Start   #############
############# Epoch 37: Validation End     #############
Epoch: 37 	Avgerage Training Loss: 0.000041 	Average Validation Loss: 0.000155
Validation loss decreased (0.000157 --> 0.000155).  Saving model ...
############# Epoch 37  Done   #############

############# Epoch 38: Training Start   #############
Epoch: 38, Training Loss:  0.02763116918504238
############# Epoch 38

### Check & Visualize validation loss
 - PID was dead at Learning rate 0.004... Evaluation is possible up to 0.003.

In [ ]:
color = ['dodgerblue', 'green', 'violet', 'orange']

In [ ]:
learning_rate = [
                 0.00001, 0.00002, 0.00003, 0.00004, 0.00005,                 
                 0.0001, 0.0002, 0.0003, 0.0004, 0.0005,
                 0.001, 0.002, 0.003]
plt.figure(figsize=(16, 10))
plt.xticks(epoch_list)

for index, lr in enumerate(learning_rate):
    plt.title("Losses per epoch")
    plt.plot(epoch_list, 
             train_losses_lr[lr], 
             #color=color[index], 
             label=f"{lr} - Train")    
    plt.plot(epoch_list, 
             val_losses_lr[lr], 
             '--', 
             #color=color[index], 
             label=f"{lr} - Val")
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(loc="upper right")

plt.show()

In [ ]:
plt.figure(figsize=(16, 10))
plt.xticks(epoch_list)
for index, lr in enumerate(learning_rate):
    plt.title("Gap of loss per epoch")
    plt.plot(epoch_list, 
             np.array(train_losses_lr[lr]) - np.array(val_losses_lr[lr]), 
             #color=color[index], 
             label=lr)    
    plt.xlabel('Epochs')
    plt.ylabel('Gap of loss')
    plt.legend(loc="upper right")

    plt.show()

In [ ]:
plt.figure(figsize=(16, 10))
plt.xticks(epoch_list)
for index, lr in enumerate(learning_rate):
    plt.title("Average losses per epoch")
    plt.plot(epoch_list, 
             avg_train_losses_lr[lr], 
             #color=color[index], 
             label=f"{lr} - Train")    
    plt.plot(epoch_list, 
             avg_val_losses_lr[lr], 
             '--', 
             #color=color[index], 
             label=f"{lr} - Val")
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(loc="upper right")

plt.show()

In [ ]:
plt.figure(figsize=(16, 10))
plt.xticks(epoch_list)
for index, lr in enumerate(learning_rate):
    plt.title("Gap of Average loss per epoch")
    plt.plot(epoch_list, 
             np.array(avg_train_losses_lr[lr]) - np.array(avg_val_losses_lr[lr]), 
             #color=color[index], 
             label=lr)    
    plt.xlabel('Epochs')
    plt.ylabel('Gap of average loss')
    plt.legend(loc="upper right")

plt.show()

- Let's see losses in detail.
- Validation loss is always bigger than tran loss but lr = 1e-0.5. So only check lr = 1e-05.

In [ ]:
plt.figure(figsize=(16, 10))
plt.xticks(epoch_list)

plt.title("Losses per epoch: lr = 1e-05")
plt.plot(epoch_list,
            val_losses_lr[1e-05],
            label = "1e-05 - Validation")
plt.plot(epoch_list,
            train_losses_lr[1e-05],
            label = "1e-05 - Train")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="upper right")

plt.show()

In [ ]:
plt.figure(figsize=(16, 10))
plt.xticks(epoch_list)
for index, lr in enumerate(learning_rate[1:]):
    plt.title("Losses per epoch: lr = [1e-05, 1e-04, 1e-03]")
    plt.plot(epoch_list, 
             train_losses_lr[lr], 
             color=color[index], 
             label=f"{lr} - Train")    
    plt.plot(epoch_list, 
             val_losses_lr[lr], 
             '--', 
             color=color[index], 
             label=f"{lr} - Val")
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(loc="upper right")

plt.show()

In [ ]:
plt.figure(figsize=(16, 10))
plt.xticks(epoch_list)
for index, lr in enumerate(learning_rate[1:]):
    plt.title("Gap of loss per epoch: lr = [1e-05, 1e-04, 1e-03]")
    plt.plot(epoch_list, 
             np.array(train_losses_lr[lr]) - np.array(val_losses_lr[lr]), 
             color=color[index], 
             label=lr)    
    plt.xlabel('Epochs')
    plt.ylabel('Gap of loss')
    plt.legend(loc="upper right")

plt.show()

In [ ]:
plt.figure(figsize=(16, 10))
plt.xticks(epoch_list)
for index, lr in enumerate(learning_rate[1:]):
    plt.title("Average losses per epoch: lr = [1e-05, 1e-04, 1e-03]")
    plt.plot(epoch_list,
             avg_train_losses_lr[lr], 
             color=color[index], 
             label=f"{lr} - Train")    
    plt.plot(epoch_list, 
             avg_val_losses_lr[lr], 
             '--', 
             color=color[index], 
             label=f"{lr} - Val")
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(loc="upper right")

plt.show()

In [ ]:
plt.figure(figsize=(16, 10))
plt.xticks(epoch_list)
for index, lr in enumerate(learning_rate[1:]):
    plt.title("Gap of Average loss per epoch: lr = [1e-05, 1e-04, 1e-03]")
    plt.plot(epoch_list, 
             np.array(avg_train_losses_lr[lr]) - np.array(avg_val_losses_lr[lr]), 
             color=color[index], 
             label=lr)    
    plt.xlabel('Epochs')
    plt.ylabel('Gap of average loss')
    plt.legend(loc="upper right")

plt.show()

__Optimal loss:  Learning rate = 1e-05, epoch = 38__
- Though it is not the least value, but optimal.

## 5-5. Evaluate model
 - Test with validation dataset
 - 1st important index: Precision
 - 2nd importand index: Recall
   - __First, get high & stable <u>Precision</u>, then improve <u>Recall</u>.__
 - And other indexes: F1 score, confusion matrix
 
 - For metrics reference here: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics

In [ ]:
from PIL import Image, ImageFile

def preprocessing(image):
    ImageFile.LOAD_TRUNCATED_IMAGES = True
    
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
    
    try:
        #transformed_image = Image.open(urlopen(image)).convert('RGB') #OR open(image)
        transformed_image = Image.open(image).convert('RGB') #OR open(image)
    except:
        return "Cannot open image"
    
    transformed_image = transform(transformed_image)
    
    return transformed_image


def inference(image, model, device):
    preprocessed_image = preprocessing(image)
    
    if preprocessed_image == "Cannot open image":
        return "Cannot open image."

    #for gpu computation
    if device.type == 'cuda':
        model.cuda()

    model.to(device)
    model.eval()

    with torch.no_grad():
        output = model(preprocessed_image.unsqueeze(0))
        final_output = torch.sigmoid(output).cpu().detach().numpy().tolist() #1*46 list in a list
        
        return final_output
    #     print(final_output)
    #     print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
#         result_pair = zip(train_df.columns[1:].to_list(), final_output[0])
#         result_dict = {}
#         for label, score in result_pair:
#             if score > 0.1: #Set prediction threshold
#                 result_dict[label] = score
#     return sorted(result_dict.items(), key=(lambda x: x[1]), reverse=True)

In [ ]:
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
#from sklearn.metrics import label_ranking_average_precision_score


def get_metrics(models_path, average_param, label_names):
    #Load model
    load_model = ResNeXt50Class(len(label_names))
    load_model = load_model.cuda() #for GPU computation
    load_model = nn.DataParallel(load_model) # Distributed
    

    y_true = valid_dataset.labels
    y_evaluations = {}

    for file in os.listdir(models_path):
        y_pred = []

        if file.split('_')[0] == 'curr' and file.split('_')[2] == '1e-05':
            model = os.path.join(models_path, file)
            # optimizer
            val_lr = float(file.split('_')[2])
            val_optimizer = make_optimizer(load_model, val_lr)            
            print(f"===========================Start evaluating model {file}, at lr {val_lr}===========================")
            #Load model --> This isn't necessary if you evaluate model right after training.
            prediction_model = load_ckp(model,
                                                        load_model,
                                                        val_optimizer)[0] #load_ckp: [model, optimizer, checkpoint['epoch'], valid_loss_min.item()]
            
            #Evaluate.
            for i in tqdm(range(len(valid_dataset))):
                prediction = np.array(inference(valid_dataset.feed_image[i],
                                                            prediction_model,
                                                            device))[0]
                prediction_over_threshold = np.where(prediction > 0.55, 1, 0).tolist()  #Threshold for each class is 0.6 #up to 0.5, Recall=1.0 
                y_pred.append(prediction_over_threshold)
                if i != 0 and i % 5000 == 0:
                    _precision_score = precision_score(y_true[:i+1], y_pred, average=average_param) #average: ['micro', 'macro', 'weighted', 'samples']
                    _recall_score = recall_score(y_true[:i+1], y_pred, average=average_param)
                    _f1_score = f1_score(y_true[:i+1], y_pred, average=average_param)
                    _average_precision_score = average_precision_score(y_true[:i+1], y_pred, average=average_param)
                    print(f'current index number: {i}')
                    print(f'current Precision: {_precision_score}')
                    print(f'current Recall: {_recall_score}')
                    print(f'current f1 score: {_f1_score}')
                    print(f'current AP: {_average_precision_score}')
                    print('\n')
                
                
            #Append evaluation result into y_preds
            _precision_score = precision_score(y_true, y_pred, average=average_param) #average: ['micro', 'macro', 'weighted', 'samples']
            _recall_score = recall_score(y_true, y_pred, average=average_param)
            _f1_score = f1_score(y_true, y_pred, average=average_param)
            _average_precision_score = average_precision_score(y_true, y_pred, average=average_param)            
            _classification_report = classification_report(y_true, y_pred, target_names = label_names)
            result_dict = {}
            result_dict['precision'] = _precision_score
            result_dict['recall'] = _recall_score
            result_dict['f1score'] = _f1_score
            result_dict['AP'] = _average_precision_score
            result_dict['classification_report'] = _classification_report
            y_evaluations[model.split('/')[-1]] = result_dict
            print(f"Number of finished model: {len(y_evaluations)}")
            print(f"===========================Done evaluating!===========================")
        else:
            pass
    
    return y_evaluations

models_path = '/home/ubuntu/Desktop/Project/autolabeler_classifier/resnext50_model/20211207/'
resnext_evaluations_for_20211207_3 = get_metrics(models_path, 'weighted', labels)

- Compare metrics

In [ ]:
#1. precision
precisions = []

for chekpoint, metrics in resnext_evaluations_for_20211207_3.items():
    precisions.append(metrics['precision'])

max_precision = max(precisions)
max_precision_index = precisions.index(max_precision)

print(f"max precision: {max_precision} | Checkpoint: {list(resnext_evaluations_for_20211207_3.keys())[max_precision_index]}")
print("===== Check if this max metric value is correct. =====")
sorted(precisions, reverse=True)

In [ ]:
#2. recall
recalls = []

for chekpoint, metrics in resnext_evaluations_for_20211207_3.items():
    recalls.append(metrics['recall'])

max_recall = max(recalls)
max_recall_index = recalls.index(max_recall)

print(f"max recall: {max_recall} | Checkpoint: {list(resnext_evaluations_for_20211207_3.keys())[max_recall_index]}")
print("===== Check if this max metric value is correct. =====")
sorted(recalls, reverse=True)

In [ ]:
#3. f1-score
f1_scores = []

for chekpoint, metrics in resnext_evaluations_for_20211207_3.items():
    f1_scores.append(metrics['f1score'])

max_f1score = max(f1_scores)
max_f1score_index = f1_scores.index(max_f1score)

print(f"max f1score: {max_f1score} | Checkpoint: {list(resnext_evaluations_for_20211207_3.keys())[max_f1score_index]}")
print("===== Check if this max metric value is correct. =====")
sorted(f1_scores, reverse=True)

In [ ]:
#4. AP
aps = []

for chekpoint, metrics in resnext_evaluations_for_20211207_3.items():
    aps.append(metrics['AP'])

max_ap = max(aps)
max_ap_index = aps.index(max_ap)

print(f"max AP: {max_ap} | Checkpoint: {list(resnext_evaluations_for_20211207_3.keys())[max_ap_index]}")
print("===== Check if this max metric value is correct. =====")
sorted(aps, reverse=True)

- f1-score, recall, AP is best at __learning rate = 1e-05, epoch=48__.
- Precision is best at __learning rate = 1e-05, epoch=45__.

- Now, compare best precision, f1-score, AP with precision, f1-score, AP of __learning rate = 1e-05, epoch=45__.
- Plus, compare best recall with recall of __learning rate = 1e-05, epoch=48__.

In [ ]:
precision_1e05_47 = resnext_evaluations_for_20211207_3['curr_ckpt_1e-05_47']['precision']
print(f"Precision of curr_ckpt_1e-05_47: {precision_1e05_47}, And (max_precision - precision_1e05_47) = {max_precision - precision_1e05_47} \n")
print(max_precision, precision_1e05_47, '\n')

f1_1e05_47 = resnext_evaluations_for_20211207_3['curr_ckpt_1e-05_47']['f1score']
print(f"F1-score of curr_ckpt_1e-05_47: {f1_1e05_47}, And (max_f1score - f1_1e05_47) = {max_f1score - f1_1e05_47} \n")
print(max_f1score, f1_1e05_47, '\n')

AP_1e05_47 = resnext_evaluations_for_20211207_3['curr_ckpt_1e-05_47']['AP']
print(f"AP of curr_ckpt_1e-05_47: {AP_1e05_47}, And (max_ap - AP_1e05_47) = {max_ap - AP_1e05_47} \n")
print(max_ap, AP_1e05_47, '\n')


recall_1e05_48 = resnext_evaluations_for_20211207_3['curr_ckpt_1e-05_48']['recall']
print(f"Recall of curr_ckpt_1e-05_48: {recall_1e05_48}, And (max_recall - recall_1e05_48) = {max_recall - recall_1e05_48} \n")
print(max_recall, recall_1e05_48, '\n')

- Gap of f1-score, recall, AP is less than gap of precision.
- So, let's take highest f1-score, recall, AP and save the metric records into a json file.

In [ ]:
import json

with open(os.path.join(models_path, 'metric_resnext50_20211207(3).json'), 'w') as f:
    json.dump(resnext_evaluations_for_20211207_3, f, ensure_ascii=False)

# 6. Inference

## 6-1. Define preprocessing function for input image.

In [ ]:
def preprocessing(image):
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
    
    try:
        transformed_image = Image.open(urlopen(image)).convert('RGB') #OR open(image)
    except:
        return "Cannot open image"
    
    transformed_image = transform(transformed_image)
    
    return transformed_image

## 6-2. Load saved model for inference

In [ ]:
#Load model
load_model = ResNeXt50Class()
load_model = load_model.cuda() #for GPU computation
load_model = nn.DataParallel(load_model) # Distributed
best_model_path = os.path.join(checkpoint_path, "20211126/curr_ckpt_1e-05_16") #currently best model state.

best_optimizer = make_optimizer("loaded_model or best_model_path", "LEARNING_RATE") # <-- Parameter shoud be changed!

predicton_model = load_ckp(best_model_path,  #Path to the saved checkpoint
                        load_model,
                        best_optimizer)[0] #load_ckp: [model, optimizer, checkpoint['epoch'], valid_loss_min.item()]

## 6-3. Define inference function
 - Return dictionaries of predicted labels: {label1: score1, label2: score2, ....}
 - __Labeles which has lower score than threshold will be ignored.__

In [ ]:
def inference(image, model, device):
    preprocessed_image = preprocessing(image)
    
    if preprocessed_image == "Cannot open image":
        return "Cannot open image."

    #for gpu computation
    if device.type == 'cuda':
        model.cuda()

    model.to(device)
    model.eval()

    with torch.no_grad():
        output = model(preprocessed_image)
        final_output = torch.sigmoid(output).cpu().detach().numpy().tolist() #1*46 list in a list
    #     print(final_output)
    #     print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
        result_pair = zip(train_df.columns[1:].to_list(), final_output[0])
        result_dict = {}
        for label, score in result_pair:
            if score > 0.1: #Set prediction threshold
                result_dict[label] = score
    return sorted(result_dict.items(), key=(lambda x: x[1]), reverse=True)

- Reference code for inference!

In [ ]:
# model.eval()
# for sample_id in [1,2,3,4,6]:
#     test_img, test_labels = test_dataset[sample_id]
#     test_img_path = os.path.join(img_folder, test_dataset.imgs[sample_id])
#     with torch.no_grad():
#         raw_pred = model(test_img.unsqueeze(0)).cpu().numpy()[0]
#         raw_pred = np.array(raw_pred > 0.5, dtype=float)

#     predicted_labels = np.array(dataset_val.classes)[np.argwhere(raw_pred > 0)[:, 0]]
#     if not len(predicted_labels):
#         predicted_labels = ['no predictions']
#     img_labels = np.array(dataset_val.classes)[np.argwhere(test_labels > 0)[:, 0]]
#     plt.imshow(Image.open(test_img_path))
#     plt.title("Predicted labels: {} \nGT labels: {}".format(', '.join(predicted_labels), ', '.join(img_labels)))
#     plt.axis('off')
#     plt.show()

## 6-4. Demo test

In [ ]:
test_image = ""

inference(test_image, #Input sentence
         predicton_model,
          device) #CPU or GPU